In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Oct 31 11:47:40 2019

@author: mor
"""

import torch
import torch.nn as nn
import torchvision
import torch.utils.data
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch import optim
from torch.utils.data.sampler import SubsetRandomSampler



In [ ]:


# =============================================================================
# OOP-based
# =============================================================================

# Create 10 random images of shape (1, 28, 28)
images = torch.rand(10, 1, 28, 28)

# Build 6 conv. filters
conv_filters = torch.nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, stride=1, padding=1)

# Convolve the image with the filters
output_feature = conv_filters(images)
print(output_feature.shape)



In [ ]:


# =============================================================================
# Functional - based
# =============================================================================

# Create 10 random images
image = torch.rand(10, 1, 28, 28)

# Create 6 filters
filters = torch.rand(6, 1, 3, 3)

# Convolve the image with the filters
output_feature = F.conv2d(image, filters, stride=1, padding=1)
print(output_feature.shape)



In [ ]:


im=torch.tensor([[[[ 8.,  1.,  2.,  5.,  3.,  1.],
                [ 6.,  0.,  0., -5.,  7.,  9.],
                [ 1.,  9., -1., -2.,  2.,  6.],
                [ 0.,  4.,  2., -3.,  4.,  3.],
                [ 2., -1.,  4., -1., -2.,  3.],
                [ 2., -4.,  5.,  9., -7.,  8.]]]])



In [ ]:


# Build a pooling operator with size `2`.     ***OOP
max_pooling = torch.nn.MaxPool2d(2)

# Apply the pooling operator
output_feature = max_pooling(im)

# Use pooling operator in the image          ***Functional
output_feature_F = F.max_pool2d(im,2)

# print the results of both cases
print(output_feature)
print(output_feature_F)



In [ ]:



# Build a pooling operator with size `2`.     ***OOP
avg_pooling = torch.nn.AvgPool2d(2)

# Apply the pooling operator
output_feature = avg_pooling(im)

# Use pooling operator in the image          ***Functional
output_feature_F = F.avg_pool2d(im, 2)

# print the results of both cases
print(output_feature)
print(output_feature_F)



In [ ]:


class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
		
        # Instantiate the ReLU nonlinearity
        self.relu = nn.ReLU()
        
        # Instantiate two convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=5, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=5, out_channels=10, kernel_size=3, padding=1)
        
        # Instantiate a max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        
        # Instantiate a fully connected layer
        self.fc = nn.Linear(7 * 7 * 10, 10)

    def forward(self, x):

        # Apply conv followd by relu, then in next line pool
        x = self.relu(self.conv1(x))
        x = self.pool(x)

        # Apply conv followd by relu, then in next line pool
        x = self.relu(self.conv2(x))
        x = self.pool(x)

        # Prepare the image for the fully connected layer
        x = x.view(-1, 7 * 7 * 10)

        # Apply the fully connected layer and return the result
        return self.fc(x)



In [ ]:

        
import torch
import torchvision
import torch.utils.data
import torchvision.transforms as transforms
from torch.utils.data import SubsetRandomSampler

# Transform the data to torch tensors and normalize it 
transform = transforms.Compose([transforms.ToTensor(),
								transforms.Normalize((0.1307,), ((0.3081,)))])

# Prepare the datasets
trainset = torchvision.datasets.MNIST('mnist', train=True, 
									  download=True, transform=transform)
testset = torchvision.datasets.MNIST('mnist', train=False, 
									  download=True, transform=transform)

# Prepare the dataloaders
train_loader = torch.utils.data.DataLoader(trainset, batch_size=32,
										  shuffle=True, num_workers=0)

sampler = SubsetRandomSampler(list(range(10)))
test_loader = torch.utils.data.DataLoader(testset, batch_size=1,sampler=sampler,
										 shuffle=False, num_workers=0)    



In [ ]:

net = Net(10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=3e-4)
net.train()


In [ ]:


for i, data in enumerate(train_loader, 0):
    inputs, labels = data
    optimizer.zero_grad()

    # Compute the forward pass
    outputs = net(inputs)
        
    # Compute the loss function
    loss = criterion(outputs,labels)
        
    # Compute the gradients
    loss.backward()
    
    # Update the weights
    optimizer.step()



In [ ]:


correct, total = 0, 0
predictions = []
net.eval()



In [ ]:


# Iterate over the data in the test_loader
for data in test_loader:
  
    # Get the image and label from data
    image, label = data
    
    # Make a forward pass in the net with your image
    output = net(image)
    
    # Argmax the results of the net
    _, predicted = torch.max(output.data, 1)
    if predicted == label:
        print("Yipes, your net made the right prediction " + str(predicted))
    else:
        print("Your net prediction was " + str(predicted) + ", but the correct label is: " + str(label))

